In [1]:
# 가로로 넓게 쓰기
from IPython.core.display import display, HTML  
display(HTML(""))

print("Setup Complete")

Setup Complete


In [2]:
import time 
import datetime
from datetime import datetime

import pandas as pd
import numpy as np
import os

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

import pandas_profiling 
from tqdm.auto import tqdm
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

In [3]:
path = "/home/kym/ML/input/kamp/대회 과제"
os.chdir(path)

df = pd.read_csv('competition_dataset.csv', encoding='cp949')
df.head()

,STD_DT,NUM,MELT_TEMP,MOTORSPEED,MELT_WEIGHT,INSP,TAG
0,2020-03-04 0:00,0,489,116,631,3.19,OK
1,2020-03-04 0:00,1,433,78,609,3.19,OK
2,2020-03-04 0:00,2,464,154,608,3.19,OK
3,2020-03-04 0:00,3,379,212,606,3.19,OK
4,2020-03-04 0:00,4,798,1736,604,3.21,OK


In [4]:
df['MELT_TEMP'] = (df['MELT_TEMP'] / 10) # 용해온도, 교반속도 데이터는 소수점 1 자리 생략 
df['MOTORSPEED'] = (df['MOTORSPEED'] / 10)

result_mapping = {
    "OK": 1,
    "NG": 0
}

df.loc[:, "TAG"] = df.TAG.map(result_mapping)
df['STD_DT'] = df['STD_DT'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d %H:%M'))
df.set_index('STD_DT', inplace=True)

In [5]:
# Heatmap

fig, ax = plt.subplots(1, 1, figsize=(20, 20))
df_corr = df.corr()

mask = np.zeros_like(df_corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

sns.heatmap(df_corr, ax=ax,
           square=True, center=0, linewidth=1,
           cmap=sns.diverging_palette(240, 10, as_cmap=True),
           cbar_kws={'shrink': .82},
           mask=mask,
           annot=True,
           annot_kws={'size':7}
           )
ax.set_title(f'Correlation', loc='left', fontweight='bold')
plt.show()

In [6]:
# melt_weight 는 종속 변수들과 상관 관계가 거의 없음. 

corr = df.corr(method='pearson')
corr

,NUM,MELT_TEMP,MOTORSPEED,MELT_WEIGHT,INSP,TAG
NUM,1.000000,0.000188,-0.000050,0.080079,-0.000524,0.086698
MELT_TEMP,0.000188,1.000000,0.944929,-0.000336,0.916295,0.310586
MOTORSPEED,-0.000050,0.944929,1.000000,0.000123,0.887813,0.264693
MELT_WEIGHT,0.080079,-0.000336,0.000123,1.000000,-0.000005,-0.012084
INSP,-0.000524,0.916295,0.887813,-0.000005,1.000000,0.272580
TAG,0.086698,0.310586,0.264693,-0.012084,0.272580,1.000000


In [7]:
df1 = df[['MELT_TEMP', 'MOTORSPEED', 'TAG']]
df1.head()

,MELT_TEMP,MOTORSPEED,TAG
STD_DT,,,
2020-03-04,48.9,11.6,1
2020-03-04,43.3,7.8,1
2020-03-04,46.4,15.4,1
2020-03-04,37.9,21.2,1
2020-03-04,79.8,173.6,1


In [8]:
split_data = int(df1.shape[0] * 0.7)

train = df1[:split_data]
test = df1[split_data:]

In [9]:
print(train.shape)
print(test.shape)

(584640, 3)
(250560, 3)


In [13]:
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from imblearn.over_sampling import SMOTE

scaler = preprocessing.MinMaxScaler()

train_sc = scaler.fit_transform(train)
test_sc = scaler.transform(test)

In [16]:
train_sc

array([[0.34541985, 0.06430155, 1.        ],
       [0.23854962, 0.04323725, 1.        ],
       [0.29770992, 0.08536585, 1.        ],
       ...,
       [0.31679389, 0.08203991, 1.        ],
       [0.1870229 , 0.03769401, 1.        ],
       [0.32251908, 0.05820399, 1.        ]])

In [17]:
x_train_values = train_sc[:, :-1]
y_train_values = train_sc[:, -1]

In [18]:
smote = SMOTE(random_state=42)

x_train_over, y_train_over = smote.fit_sample(x_train_values, y_train_values)
print("SMOTE 적용 전 데이터 셋: ", x_train_values.shape, y_train_values.shape)
print("SMOTE 적용 후 데이터 셋: ", x_train_over.shape, y_train_over.shape)
print("SMOTE 적용 후 LABEL 분포: \n", pd.Series(y_train_over).value_counts())

SMOTE 적용 전 데이터 셋:  (584640, 2) (584640,)
SMOTE 적용 후 데이터 셋:  (821032, 2) (821032,)
SMOTE 적용 후 LABEL 분포: 
 1.0    410516
0.0    410516
dtype: int64


In [19]:
def make_dataset(data, label, window_size):
    feature_list = []
    label_list = []
    
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i : i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)